<a href="https://colab.research.google.com/github/rizaho/ml_for_financial_analysis/blob/main/201_4_FACEBOOK_PROPHET.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#